# Task - Part 1
* Eylon Yehiel - 209270180
* Michael Mutter - 315970194

* Project GitHub: https://github.com/EylonYehiel/ML-and-Data-mining

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# defining initial soup.
main_url = "https://www.ad.co.il/car"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}


try:
    html = requests.get(main_url, headers)
    print("Successful requset")
except:
    print("Invalid URL, please check and try again")
content = html.content
soup = BeautifulSoup(content, 'html.parser')

Successful requset


In [3]:
# Function for getting al specific cars by user's keyword
car_type = str(input("Which car model data would you like to get? "))
def get_model_links(soup):
    desired_links = []
    for car in soup('div',class_="card-body p-md-3"): # Runnning along the cars of each page if the keyword in its nams.
        s_link = car.find('a').get('href')
        if car_type in car.find('h2').get_text() and s_link not in desired_links:
            desired_links.append(s_link)
        else:
            continue
    return desired_links

Which car model data would you like to get? אאודי


In [4]:
# A loop in favor of getting the urls from all pages and insert them into one list.
page = 1
pages_num = int(input("How much pages to run on? ")) # Option of choosing less pages to scrap
all_cars_links = []
while True:  # Runnning along pages till break
    idx_url = "https://www.ad.co.il/car?pageindex=" + str(page)
    html = requests.get(idx_url)
    content = html.content
    soup = BeautifulSoup(content, 'html.parser')
    relevant = get_model_links(soup)
    all_cars_links.extend(relevant) # Our previous function
    page += 1 # Move to the next page
    if page > pages_num:
        break
    
# Interactive car-manufacturer selection and  with 'input' user feeding.

How much pages to run on? 94


In [5]:
print("Amount of requested cars: ", len(all_cars_links), "\n")
print(all_cars_links) # ID's of all desired car model saved as 'idx_url' 

Amount of requested cars:  120 

['/ad/16138498', '/ad/16029455', '/ad/16018151', '/ad/15952719', '/ad/15952738', '/ad/15952715', '/ad/15952731', '/ad/15952704', '/ad/15952721', '/ad/15952690', '/ad/15939168', '/ad/15935101', '/ad/16042006', '/ad/15824044', '/ad/15575264', '/ad/15504094', '/ad/15414791', '/ad/15407642', '/ad/15215519', '/ad/15106302', '/ad/15038482', '/ad/15001357', '/ad/14864229', '/ad/14786959', '/ad/14526825', '/ad/14679860', '/ad/14655342', '/ad/14626680', '/ad/14617912', '/ad/14614278', '/ad/14611331', '/ad/14608106', '/ad/14603452', '/ad/14566452', '/ad/14530828', '/ad/14522325', '/ad/14514816', '/ad/14510702', '/ad/14508530', '/ad/14494214', '/ad/14489931', '/ad/14486277', '/ad/14460505', '/ad/14436368', '/ad/14424778', '/ad/14414765', '/ad/14408617', '/ad/14394994', '/ad/14394989', '/ad/14390701', '/ad/14381635', '/ad/14364706', '/ad/14348848', '/ad/14337895', '/ad/14303912', '/ad/14264320', '/ad/14261057', '/ad/14227495', '/ad/14227677', '/ad/14227617', '/ad/1

In [6]:
final = []

for identifier in all_cars_links: # Runnning along requested cars
    full_url = "https://www.ad.co.il" + identifier
    try:
        html = requests.get(full_url, headers)
        content = html.content
        soup = BeautifulSoup(content, 'html.parser')
        table = soup.find('div', class_ = "col-xxl-4 col-lg-4 col-md-5")
        table2 = soup.find('div', class_ = "col-xxl-8 col-lg-8 col-md-7")

        car_att_dict = {}
        car_att_dict['Car_name'] = table.find('div',class_ = "d-flex justify-content-between")('h2')[0].get_text()

        try:
            car_att_dict['Price'] = table.find('div',class_ = "d-flex justify-content-between")('h2')[1].get_text()
        except IndexError:
            car_att_dict['Price'] = None

        for a in table.find_all('tr'):  # Loop for extraction of relevant attributes and puttnig them in small dictionary for every car.
            key = a.find_all('td')[0].get_text().strip()
            value = a.find_all('td')[1].get_text().strip() 
            car_att_dict[key] = value

        car_att_dict['Pic_num'] = len(table2('figure'))
        try:
            car_att_dict['Description'] = table2.find('p').get_text().strip()
        except AttributeError:
            car_att_dict['Description'] = None
        car_att_dict['Cre_date'] = table2.find_all('div', 'px-3')[0].get_text()[-10:]
        car_att_dict['Repub_date'] = table2.find_all('div', 'px-3')[1].get_text()[-10:]
        
        final.append(car_att_dict)
        
    except:
        print("Invalid URL, please check and try again")
    
    

In [7]:
data = pd.DataFrame(final)

<IPython.core.display.Javascript object>

### Cast and manipulate:

In [8]:
# Renaming:
data.rename(columns = {'יד': 'Hand',
                       'שנה': 'Year',
                       'ת. הילוכים': 'Gear',
                       'נפח': 'Engine_capacity',
                       'סוג מנוע': 'Engine_type',
                       'ק"מ': 'Km',
                       'צבע': 'Color',
                       'בעלות קודמת': 'Prev_ownership',
                       'בעלות נוכחית': 'Curr_ownership',
                       'אזור': 'Area',
                       'עיר': 'City',
                       'טסט עד': 'Test'},
              inplace=True)
data

,Car_name,Price,Year,Hand,Gear,Engine_capacity,Engine_type,Km,Color,Prev_ownership,Curr_ownership,Area,City,Pic_num,Description,Cre_date,Repub_date,Test
0,אאודי A5,"68,000 ₪",2013,3,אוטומטית,"1,800",בנזין,"119,000",שחור,פרטית,פרטית,ירושלים והסביבה,ירושלים,5,‎!!קילומטר נמוך !! רכב במצב תצוגה טיפול גדול ח...,17/03/2024,31/03/2024,NaN
1,אאודי A1,"55,000 ₪",2016,2,אוטומטית,"1,000",בנזין,"135,000",שחור,ליסינג,פרטית,NaN,בית ג'ן,3,רכב שמור מטופל בזמן יש עליו כל מיני תוספות מי ...,25/11/2023,24/11/2023,NaN
2,אאודי A1,"122,000 ₪",2021,1,אוטומטית,"1,000",בנזין,NaN,שחור,פרטית,פרטית,ירושלים והסביבה,ירושלים,9,רכב אאודי A1 דגם סטייל קומפורט \nאחרי טיפול של...,11/11/2023,11/11/2023,NaN
3,אאודי RS5,"65,000 ₪",2019,4,אוטומטית,"4,500",גז,"7,000",אפור,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,4,אאודי RS5 ספורטבק 2019 כמעט חדשה בדייטונה גריי...,02/10/2023,02/10/2023,NaN
4,אאודי RS5,"65,000 ₪",2019,4,אוטומטית,"4,500",גז,"7,000",אפור,פרטית,פרטית,חולון - בת ים,חולון,4,נתניהאאודי RS5 ספורטבק 2019 כמעט חדשה בדייטונה...,02/10/2023,02/10/2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,אאודי 100,"1,000,000 ₪",2023,1,אוטומטית,"1,300",בנזין,NaN,אפור,פרטית,חברה,תל אביב,תל אביב יפו,0,"דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...",13/11/2023,13/11/2023,NaN
116,אאודי 80,"1,000,000 ₪",2022,1,אוטומטית,800,דיזל,NaN,אדום,פרטית,חברה,תל אביב,תל אביב יפו,0,"דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...",07/11/2023,07/11/2023,NaN
117,אאודי 80,"1,000 ₪",2023,1,אוטומטית,"2,300",בנזין,NaN,אדום,פרטית,פרטית,תל אביב,תל אביב יפו,0,"דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...",01/11/2023,01/11/2023,12/2024
118,אאודי A1,42 ₪,2014,6,אוטומטית,"2,100",בנזין,"147,000",אפור,פרטית,פרטית,תל אביב,תל אביב יפו,0,"דרושים דרושות לעבודה מהבית בקלדנות, כתיבה, הזנ...",18/09/2022,18/09/2022,NaN


In [9]:
# Casting:
data['Cre_date'] = pd.to_datetime(data['Cre_date'], dayfirst = True)
data['Repub_date'] = pd.to_datetime(data['Repub_date'], dayfirst = True)
data['Gear'] = pd.Categorical(data['Gear'])
data['Engine_type'] = pd.Categorical(data['Engine_type'])
data['Prev_ownership'] = pd.Categorical(data['Prev_ownership'])
data['Curr_ownership'] = pd.Categorical(data['Curr_ownership'])
data['Year'] = data['Year'].astype('Int32')
data['Hand'] = data['Hand'].astype('Int32')
data['Pic_num'] = data['Pic_num'].astype('Int32')
data['Engine_capacity'] = data['Engine_capacity'].str.replace(',', '').astype('Int32')
data['Km'] = data['Km'].str.replace(',', '').astype('Int32')
data['Price'] = data['Price'].str.replace(r'[^\d]', '', regex = True).astype('float') # Extract only the digits using regular expressions

data['Test'] = data['Test'].apply(lambda x: "30/" + x if pd.notna(x) else x)
data['Test'] = pd.to_datetime(data['Test'], dayfirst = True)

# Calculate the time difference between today and future test expire date
today = pd.to_datetime('today')
data['Test'] = (data['Test'] - today).dt.days.astype('Int32')

# Renaming for identical column names, will be helpful on supply score manipulation
data = data.rename(columns={'Car_type': 'Type'})

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# Removing missing data of cars with no price

data = data.dropna(subset=['Price'])

In [11]:
# Fake ads need to be be removed

fakes = data[data['Description'].str.contains('דרושים|משרד', na=False)]
fakes['Description']

94     לעבודה מהבית דרושים דרושות, להקלדה, הזנת נתוני...
103    דרושים דרושות לעבודה מהבית בקלדנות, כתיבה, הזנ...
111                                         עבודה משרדית
114    דרושים/ות לעבודה בהקלדת נתונים, מדובר על עבודה...
115    דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...
116    דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...
117    דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...
118    דרושים דרושות לעבודה מהבית בקלדנות, כתיבה, הזנ...
119    דרושים! לעבודה באקסל, מדובר על עבודה בהזנת תוכ...
Name: Description, dtype: object

In [12]:
# Everything except fakes
data = data[~data['Description'].str.contains('דרושים|משרד', na=False)]

In [13]:
# Also cars with excessively unreasonable low prices need to be removed (?)
# data[data['Price'] < 5000]

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 113
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Car_name         100 non-null    object        
 1   Price            100 non-null    float64       
 2   Year             100 non-null    Int32         
 3   Hand             100 non-null    Int32         
 4   Gear             100 non-null    category      
 5   Engine_capacity  100 non-null    Int32         
 6   Engine_type      100 non-null    category      
 7   Km               93 non-null     Int32         
 8   Color            54 non-null     object        
 9   Prev_ownership   54 non-null     category      
 10  Curr_ownership   54 non-null     category      
 11  Area             88 non-null     object        
 12  City             100 non-null    object        
 13  Pic_num          100 non-null    Int32         
 14  Description      97 non-null     object   

---
---

#### Supply score integration:

In [15]:
import json

In [16]:
url = "https://data.gov.il/api/3/action/datastore_search?resource_id=5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6"
response = requests.get(url)
try:
    from_gov = json.loads(response.content)
    from_gov = from_gov['result']['records']
except:
    print("Data in update progress, please try again later")

In [17]:
pd.DataFrame(from_gov).sample(5)

<IPython.core.display.Javascript object>

,_id,sug_degem,tozeret_cd,tozeret_nm,tozeret_eretz_nm,tozar,degem_cd,degem_nm,shnat_yitzur,mispar_rechavim_pailim,mispar_rechavim_le_pailim,kinuy_mishari
28359,28360,P,143,ב מ וו גרמניה,גרמניה,ב מ וו,838,7D01,2019,10,1,740E IPERFORMAN
31162,31163,M,301,פולקסווגן פולי,פולין,פולקסווגן,182,7FD131,2011,17,1,NEW TRANSPORTER
29769,29770,P,19,אאודי,גרמניה,אאודי,661,8D20LK,1999,0,3,A4
15674,15675,P,715,פולקסווגן מכסי,מקסיקה,פולקסווגן,30,1K20E1,2008,6,7,JETTA
10403,10404,P,19,אאודי,גרמניה,אאודי,73,8PAABG,2012,123,26,A3 SPORTBACK


In [18]:
# Create from the API a dataframe for more convinient manipulation
temp_list = []
for item in from_gov:
    if item ['tozar'] == car_type:
        temp_dict = {}
        temp_dict['Type'] = item['kinuy_mishari']
        temp_dict['Year'] = item['shnat_yitzur']
        temp_dict['Supply_score'] = item['mispar_rechavim_pailim']       
        temp_list.append(temp_dict)
data2 = pd.DataFrame(temp_list)
data2

<IPython.core.display.Javascript object>

,Type,Year,Supply_score
0,A4,2003,3
1,A6,2004,6
2,A6,2006,14
3,A4,2017,32
4,Q5,2016,17
...,...,...,...
1601,A1,2012,86
1602,A4,2007,4
1603,A1 SPORTBACK,2021,40
1604,A1,2011,135


In [19]:
# Display the discrepancy between names
types = data.Car_name.unique()
types2 = data2.Type.unique()

print('Our models: \n',types, '\n')
print('External models:\n ', types2)

# We can notice the two sources are not similar 

Our models: 
 ['אאודי A5' 'אאודי A1' 'אאודי RS5' 'אאודי A6' 'אאודי A3' 'אאודי 100'
 'אאודי TT' 'אאודי A4' 'אאודי Q3' 'אאודי RS3' 'אאודי A7' 'אאודי All Road'
 'אאודי S3' 'אאודי S5' 'אאודי TT קופה' 'אאודי 80' 'אאודי S8' 'אאודי S7'
 'אאודי R8'] 

External models:
  ['A4' 'A6' 'Q5' 'Q7' 'Q3' 'Q2' 'Q5 DESIGN' 'A8' 'A5 COUPE' 'A3 DESIGN'
 'A3 SEDAN' 'TT' 'AUDI TT' 'A1' 'AUDI Q7' 'AUDI S3' 'A7' 'A3'
 'A5 SPORTBACK' 'A5' 'RS5' 'Q3 SPORTBACK' 'RSQ3 SPORTBACK'
 'אודי 3A 8.1 אוטומטי' 'A1 SPORTBACK' 'אוטומטי' "אודי 6-Aפרטי רג' 4דל"
 'SPORTBACK A3' 'S3 SEDAN' 'RS5 COUPE' 'SQ5' 'S3' 'A6 2.8Q'
 'אודי 3A T 8.1 5הילוכ' '5 הילוכים' 'A3 SPORTBACK' 'SQ5 SPORTBACK'
 'אוטומטי 4 הילוכים' 'S6' 'S8' 'S3 SPORTBACK' 'TT ROADSTER' 'Q5 SPORTBACK'
 'AUDI A4' 'SQ7' 'AUDI A3' 'AUDI A5' 'S5' 'Q3 SBACK ETRON'
 "אודי IDT 9.1 אוט' 4A" 'AUDI S8' 'ALLROAD' 'RSQ3' 'אודי 4A 8.2 קואטרו K'
 'Q7 50TDI' 'ETRON SPORTBACK' 'S5 SPORTBACK' 'RS3' '4A20Y8'
 'אודי 4A טורבו 4 דלתו' 'Q8' 'A4 TURBO' 'A1 SPORTBCK' 'RSQ3 PERFORMANC'
 "אודי 

In [20]:
# Function for increasing the similarity between the datasets using regular expressions.
def extract_simple_models(models):
    simple_models = []
    for model in models:
        match = re.match(r'([A-Za-z]+\d*)', model) # 
        if match:
            simple_models.append(match.group())
        else:
            simple_models.append(None)
    return simple_models

# Extracting unique simple car models
unique_simple_models = extract_simple_models(data2.Type)
# changing the car model names to more simple names
data2['Type'] = unique_simple_models

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
# Fixing specific minor inconsistencies

data['Type'] = data['Car_name'].str.replace(car_type+" ", '')
data['Type'] = data['Type'].str.replace(" קופה", "")
data['Type'] = data['Type'].str.replace("All Road", "ALLROAD")
data['Type'] = data['Type'].str.replace(" ", "")

In [22]:
types = data.Type.unique()
types2 = data2.Type.unique()

print('Our models: \n',types, '\n')
print('External models:\n ', types2)

# Much better

Our models: 
 ['A5' 'A1' 'RS5' 'A6' 'A3' '100' 'TT' 'A4' 'Q3' 'RS3' 'A7' 'ALLROAD' 'S3'
 'S5' '80' 'S8' 'S7' 'R8'] 

External models:
  ['A4' 'A6' 'Q5' 'Q7' 'Q3' 'Q2' 'A8' 'A5' 'A3' 'TT' 'AUDI' 'A1' 'A7' 'RS5'
 'RSQ3' None 'SPORTBACK' 'S3' 'SQ5' 'S6' 'S8' 'SQ7' 'S5' 'ALLROAD' 'ETRON'
 'RS3' 'Q8' 'S4' 'RODSTR' 'S1' 'RS6' 'Q4' 'TTRS' 'R8' 'S7' 'RS7' 'QUATTRO'
 'E' 'TTS']


In [23]:
# Representing supply score for each car model by years
grouped = data2.groupby(['Type','Year']).sum().reset_index()
grouped

,Type,Year,Supply_score
0,A1,2010,31
1,A1,2011,282
2,A1,2012,315
3,A1,2013,269
4,A1,2014,139
...,...,...,...
302,TT,2021,62
303,TT,2022,4
304,TT,2023,27
305,TTRS,2022,0


In [24]:
# merging with the grouped data for getting the supply score per model per year
new_data = pd.merge(data, grouped, on=['Type', 'Year'], how='left')
new_data['Supply_score'] = new_data['Supply_score'].astype('Int32')

<IPython.core.display.Javascript object>

In [25]:
new_data

,Car_name,Price,Year,Hand,Gear,Engine_capacity,Engine_type,Km,Color,Prev_ownership,Curr_ownership,Area,City,Pic_num,Description,Cre_date,Repub_date,Test,Type,Supply_score
0,אאודי A5,68000.0,2013,3,אוטומטית,1800,בנזין,119000,שחור,פרטית,פרטית,ירושלים והסביבה,ירושלים,5,‎!!קילומטר נמוך !! רכב במצב תצוגה טיפול גדול ח...,2024-03-17,2024-03-31,<NA>,A5,30
1,אאודי A1,55000.0,2016,2,אוטומטית,1000,בנזין,135000,שחור,ליסינג,פרטית,NaN,בית ג'ן,3,רכב שמור מטופל בזמן יש עליו כל מיני תוספות מי ...,2023-11-25,2023-11-24,<NA>,A1,670
2,אאודי A1,122000.0,2021,1,אוטומטית,1000,בנזין,<NA>,שחור,פרטית,פרטית,ירושלים והסביבה,ירושלים,9,רכב אאודי A1 דגם סטייל קומפורט \nאחרי טיפול של...,2023-11-11,2023-11-11,<NA>,A1,237
3,אאודי RS5,65000.0,2019,4,אוטומטית,4500,גז,7000,אפור,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,4,אאודי RS5 ספורטבק 2019 כמעט חדשה בדייטונה גריי...,2023-10-02,2023-10-02,<NA>,RS5,4
4,אאודי RS5,65000.0,2019,4,אוטומטית,4500,גז,7000,אפור,פרטית,פרטית,חולון - בת ים,חולון,4,נתניהאאודי RS5 ספורטבק 2019 כמעט חדשה בדייטונה...,2023-10-02,2023-10-02,<NA>,RS5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,אאודי A3,93000.0,2016,2,אוטומטית,1400,בנזין,110000,לבן פנינה,פרטית,פרטית,חיפה וחוף הכרמל,טירת כרמל,9,רכב שמור ללא תיקון צבע כל התוספות חסכוני אמין ...,2022-01-27,2022-01-27,<NA>,A3,602
96,אאודי A6,165000.0,2016,2,טיפטרוניק,2000,בנזין,120000,לבן מטאלי,פרטית,פרטית,NaN,עראבה,0,אודי a6 לקצרי. מטופלת בקנאות שמורה מאוד. חיצונ...,2022-06-12,2022-06-12,<NA>,A6,35
97,אאודי A4,25000.0,2009,2,טיפטרוניק,1800,בנזין,200000,NaN,NaN,NaN,באר שבע והסביבה,אופקים,0,אאודי שמורה מאוד מצב חדש \r\nמתחייב בבדיקה מלא...,2022-06-10,2022-06-10,<NA>,A4,173
98,אאודי A6,155000.0,2015,3,אוטומטית,3000,בנזין,90000,אדום,פרטית,פרטית,נתניה והסביבה,נתניה,0,Alex,2022-05-01,2022-05-01,<NA>,A6,90


In [26]:
path_to_create_CSV = r"C:\Users\elon2\OneDrive\Desktop\Audis.csv"
new_data.to_csv(path_to_create_CSV, index = False, encoding = 'utf-8-sig')

---